<a href="https://colab.research.google.com/github/chominyeong/D.D.D/blob/main/modify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2

In [ ]:
import os
from PIL import Image
import numpy as np
from IPython.display import Image
import torch
import PIL

In [ ]:
dir_path="/content/drive/MyDrive/2기 딥러닝 프로젝트/train"
labels = ['Closed', 'Open']
x_data = []
y_data = []
for label in labels:
   path = os.path.join(dir_path, label)
   class_num = labels.index(label)
   import torchvision.transforms as transforms
   tf = transforms.ToTensor()
   for img in os.listdir(path): # closed 폴더 안에 _1.jpg, open 폴더 안에 1.jpg
       paths = os.path.join(path, img)
       img =  PIL.Image.open(paths)
       re_img = img.resize((64,64))
       re_img = tf(re_img)
       re_img = re_img.tolist()
       if len(re_img) == 3:
         x_data.append(re_img)
         y_data.append(class_num)

x_data = torch.tensor(x_data)
y_data = torch.tensor(y_data)

In [ ]:
x_data.shape

torch.Size([1449, 3, 64, 64])

In [ ]:
y_data.shape

torch.Size([1449])

In [ ]:
y_data = np.array(y_data)
y_data = torch.tensor(y_data)
y_data.shape

torch.Size([1449])

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)

In [ ]:
x_train.shape

torch.Size([1159, 3, 64, 64])

In [ ]:
y_train.shape

torch.Size([1159])

## **train**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
import torch.optim as optim

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(4096, 512)
        self.fc2 = nn.Linear(512, 1)

        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)),2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = F.max_pool2d(F.relu(self.conv3(x)), 2)
        x = x.reshape(-1, 4096)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

x = torch.ones([1,3,64,64])
conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
output1 = conv1(x)
print(output1.shape)
x1 = F.max_pool2d(F.relu(output1),2)
print(x1.shape)
conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
output2 = conv2(x1)
print(output2.shape)
x2 = F.max_pool2d(F.relu(output2), 2)
print(x2.shape)
conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
output3 = conv3(x2)
print(output3.shape)
x3 = F.max_pool2d(F.relu(output3), 2)
print(x3.shape)

torch.Size([1, 16, 64, 64])
torch.Size([1, 16, 32, 32])
torch.Size([1, 32, 32, 32])
torch.Size([1, 32, 16, 16])
torch.Size([1, 64, 16, 16])
torch.Size([1, 64, 8, 8])


In [ ]:
model = Net()
x = torch.ones(1, 3,64,64)
x = x.float()
output = model(x)
output.shape

torch.Size([1, 1])

In [ ]:
model = Net()
model.to('cuda')
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

epochs = 30

In [ ]:
def accuracy(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum / y_test.shape[0]
    acc = torch.round(acc * 100)

    return acc

In [ ]:
for epoch in range(epochs):
    running_loss = 0.0
    running_acc = 0.0

    model.train()

    for i,data in enumerate(x_train, 0):
        optimizer.zero_grad()

        data = data.to('cuda')
        labels = y_train.to('cuda')

        k = torch.tensor(labels[i])
        k = k.view(1,-1)
        k = k.float()
        k = k.to('cuda')
        outputs = model(data)
        #print("intpus_1 shape: ",input_1.shape)
        #print("outputs shape: ",outputs.shape)
        #print("labels shape",labels.shape)
         #labels = labels.view(len(labels),-1)
        loss = criterion(outputs, k)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        running_acc += accuracy(outputs, k)

        if i % 80 == 79:
            print('epoch: [%d/%d] train_loss: %.5f train_acc: %.5f' % (
                epoch + 1, epochs, running_loss / 80, running_acc / 80))
            running_loss = 0.0

print("learning finish")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]


epoch: [1/50] train_loss: 0.69876 train_acc: 48.75000
epoch: [1/50] train_loss: 0.69525 train_acc: 97.50000
epoch: [1/50] train_loss: 0.68318 train_acc: 156.25000
epoch: [1/50] train_loss: 0.67837 train_acc: 220.00000
epoch: [1/50] train_loss: 0.65744 train_acc: 287.50000
epoch: [1/50] train_loss: 0.56549 train_acc: 368.75000
epoch: [1/50] train_loss: 0.46055 train_acc: 451.25000
epoch: [1/50] train_loss: 0.31957 train_acc: 540.00000
epoch: [1/50] train_loss: 0.25567 train_acc: 630.00000
epoch: [1/50] train_loss: 0.22548 train_acc: 722.50000
epoch: [1/50] train_loss: 0.13178 train_acc: 820.00000
epoch: [1/50] train_loss: 0.09688 train_acc: 916.25000
epoch: [1/50] train_loss: 0.16200 train_acc: 1007.50000
epoch: [1/50] train_loss: 0.19212 train_acc: 1098.75000
epoch: [2/50] train_loss: 0.18044 train_acc: 96.25000
epoch: [2/50] train_loss: 0.30371 train_acc: 185.00000
epoch: [2/50] train_loss: 0.15079 train_acc: 278.75000
epoch: [2/50] train_loss: 0.17988 train_acc: 372.50000
epoch: [2/5

# **test**

In [ ]:
with torch.no_grad():
    total_acc = 0.0
    acc = 0.0
    data = x_test.to('cuda')
    labels = y_test.to('cuda')

    for i,data in enumerate(x_test, 0):
        data = data.to('cuda')
        labels = y_test.to('cuda')
        k = torch.tensor(labels[i])
        k = k.view(1,-1)
        k = k.float()
        k = k.to('cuda')
        outputs = model(data)
        acc = accuracy(outputs, k)
        total_acc += acc

        count = i

    print('avarage acc: %.5f' % (total_acc/count),'%')

print('test finish!')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.


avarage acc: 97.92388 %
test finish!


In [ ]:
def predict(pred):
  pred = pred
  outputs = model(pred)
  pred_tag = torch.round(torch.sigmoid(outputs))
  #pred_tag = torch.sigmoid(outputs)
  return pred_tag

In [ ]:
def predicts(pred):
  pred = pred
  outputs = model(pred)
  pred_tag = torch.sigmoid(outputs)
  return pred_tag